In [ ]:
import subprocess

import matplotlib.pyplot as plt
import arrow
import pandas as pd
from bs4 import BeautifulSoup
from path_helpers import path

%matplotlib inline

def is_number(s):
    """Returns True if string is a number."""
    return s.replace('.','',1).isdigit()                

def format_fields(input_list):
    fields = [x.strip(':') for x in input_list if x.find('</br>') == -1]
    return [float(x) if is_number(x) else x for x in fields]

def get_summary(soup):
    def find_seq_id(tag):
        return tag.name == u'div' and tag.decode().find('SEQ-ID') >=0

    def find_account_summary(tag):
        return tag.name == u'span' and tag.decode().find('Your Account Summary') >=0

    summary_fields = format_fields(soup.find_all(find_account_summary)[0].contents)
    summary_data = format_fields(soup.find_all(find_seq_id)[0].next_sibling.contents[0].contents)
    return dict(zip(summary_fields[1:], summary_data))

def get_water_consumption(soup):
    def find_total_consumption(tag):
        return tag.name == u'div' and tag.decode().find('Total Consumption') >= 0

    water_div = soup.find_all(find_total_consumption)[0]
    water_fields = format_fields(water_div.contents[0].contents)
    water_data = format_fields(water_div.next_sibling.next_sibling.contents[0].contents)
    return dict(zip(water_fields, water_data))

def get_water_and_sewer_charges(soup):
    def find_water_consumption(tag):
        return tag.name == u'div' and tag.decode().find('Consumption') >=0 and tag.decode().find('Total Consumption') == -1

    water_div = soup.find_all(find_consumption)[0]
    water_type = format_fields(water_div.next_sibling.contents[0])

    result = {u'Time period': water_type[0]}
    water_type = water_type[1:]
    
    consumption = format_fields(water_div.next_sibling.next_sibling.contents[0])
    rates = format_fields(water_div.next_sibling.next_sibling.next_sibling.next_sibling.contents[0])
    charges = format_fields(water_div.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.contents[0])
    
    for x in range(len(water_type)):
        result[water_type[x]] = {u'Consumption': consumption[x],
                                 u'Rate': rates[x],
                                 u'Charges': charges[x]}
    return result

def get_water_charges(soup):
    data = get_water_and_sewer_charges(soup)
    types = ['Water', 'Sewer']
    return dict(zip(types, [data[x]['Charges'] for x in types]))

def get_water_rates(soup):
    data = get_water_and_sewer_charges(soup)
    types = ['Water', 'Sewer']
    return dict(zip(types, [data[x]['Rate'] for x in types]))

def get_gas_consumption(soup):
    def find_total_consumption(tag):
        return tag.name == u'div' and tag.decode().find('Total Consumption') >= 0

    gas_div = soup.find_all(find_total_consumption)[1]
    gas_fields = format_fields(gas_div.contents[0].contents)
    gas_data = format_fields(gas_div.next_sibling.next_sibling.contents[0].contents)
    return dict(zip(gas_fields, gas_data))

def get_gas_charges(soup):
    def find_gas_rates(tag):
        return tag.name == u'div' and tag.decode().find('Gas Fixed Delivery Charge') >=0

    gas_div = soup.find_all(find_gas_rates)[0]

    gas_fields = format_fields(gas_div.contents[0])
    result = {u'Time period': gas_fields[0]}
    
    gas_fields = gas_fields[1:]
    gas_rates = format_fields(gas_div.next_sibling.next_sibling.next_sibling.contents[0])
    gas_charges = format_fields(gas_div.next_sibling.next_sibling.next_sibling.next_sibling.contents[0])
        
    result.update(dict(zip(gas_fields, gas_charges)))
    return result
        
def get_gas_rates(soup):
    def find_gas_rates(tag):
        return tag.name == u'div' and tag.decode().find('Gas Fixed Delivery Charge') >=0

    gas_div = soup.find_all(find_gas_rates)[0]
    gas_fields = format_fields(gas_div.contents[0]) 
    gas_fields = gas_fields[1:]
    gas_rates = format_fields(gas_div.next_sibling.next_sibling.next_sibling.contents[0])
    gas_charges = format_fields(gas_div.next_sibling.next_sibling.next_sibling.next_sibling.contents[0])
    
    return dict(zip([x + ' Rate' for x in gas_fields if x.find('HST') == -1 and x.find('Fixed') == -1], gas_rates))

def process_pdf(pdf_file):
    html_file = path(pdf_file.namebase + '.html')
    subprocess.check_output(['python', '%CONDA_PREFIX%\Scripts\pdf2txt.py', '-o%s' % html_file, pdf_file.name], shell=True)

    with open(html_file, 'r') as f:
        soup = BeautifulSoup(f, 'html.parser')

    html_file.remove()
    summary = get_summary(soup)
    
    if 'Pre-authorized Withdrawal' in summary.keys():
        amount_due = summary['Pre-authorized Withdrawal']
    elif 'Total Due' in summary.keys():
        amount_due = summary['Total Due']
    else:
        print "Couldn't find amount due!"
        amount_due = None

    new_name = '%s-Kitchener utilities-$%s.pdf' % (arrow.get(summary['Issue Date'], 'MMM DD YYYY').format('YYYY-MM-DD'),
        amount_due)
    pdf_file.rename(new_name)
    
    return {'summary': summary,
            'water consumption': get_water_consumption(soup),
            #'water and sewer charges': get_water_and_sewer_charges(soup),
            'gas consumption': get_gas_consumption(soup),
            'gas charges': get_gas_charges(soup),
            'gas rates': get_gas_rates(soup)
           }
        
data = []
for pdf_file in path('.').glob('*.pdf'):
    print pdf_file
    data.append(process_pdf(pdf_file))

In [ ]:
def convert_to_df(data):
    cols = data[0]['summary'].keys()
    if 'Pre-authorized Withdrawal' in cols:
        cols.remove('Pre-authorized Withdrawal')
        cols.append('Total Due')
    
    for x in data:
        if 'Pre-authorized Withdrawal' in x['summary'].keys():
            x['summary']['Total Due'] = x['summary']['Pre-authorized Withdrawal']
    data_sets = []
    for col in cols:
        data_sets.append([x['summary'][col] for x in data])
    return pd.DataFrame(data=dict(zip(cols, data_sets)))

df = convert_to_df(data)
df['Issue Date'] = [arrow.get(x, 'MMM DD YYYY').date() for x in df['Issue Date']]
df = df.set_index('Issue Date')

water_consumption = [x['water consumption']['Total Consumption'] if 'Total Consumption' in x['water consumption'] else None for x in data]
gas_consumption = [x['gas consumption']['Total Consumption'] if 'Total Consumption' in x['gas consumption'] else None for x in data]

df['Water Consumption'] = water_consumption
df['Gas Consumption'] = gas_consumption


plt.figure(); df['Gas & Water Charges'].plot(); plt.ylabel('\$')
plt.figure(); df['Gas Consumption'].plot(); plt.ylabel('m$^3$')
plt.figure(); df['Water Consumption'].plot(); plt.ylabel('m$^3$')

df